In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from keras import layers
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from nltk.tokenize import word_tokenize
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving urls_dataset.csv to urls_dataset.csv


In [ ]:
data = pd.read_csv(r"urls_dataset.csv")
print(data)

                                                     url  label
0                  http://user57245.vs.speednames.com/-/      1
1      http://morningdiaries.com/css/account-limited/...      1
2                 http://keviso.xyz/ScreenDrop/index.php      1
3      http://hosseinabyaran.ir/wp-includes/in/update...      1
4      https://sites.google.com/site/eror404safety/?p...      1
...                                                  ...    ...
20123  https://stainupurworejo.ac.id/wp-includes/css/...      1
20124             http://ceftus.org/tag/paradise-papers/      0
20125    http://transaction-authorization.droppages.com/      1
20126  http://fujoho.jp/index.php?p=shop_repo_list&id...      0
20127                    http://jayamahehe.blogspot.com/      1

[20128 rows x 2 columns]


In [ ]:
urls = data.iloc[:,0]
print(urls)

0                    http://user57245.vs.speednames.com/-/
1        http://morningdiaries.com/css/account-limited/...
2                   http://keviso.xyz/ScreenDrop/index.php
3        http://hosseinabyaran.ir/wp-includes/in/update...
4        https://sites.google.com/site/eror404safety/?p...
                               ...                        
20123    https://stainupurworejo.ac.id/wp-includes/css/...
20124               http://ceftus.org/tag/paradise-papers/
20125      http://transaction-authorization.droppages.com/
20126    http://fujoho.jp/index.php?p=shop_repo_list&id...
20127                      http://jayamahehe.blogspot.com/
Name: url, Length: 20128, dtype: object


In [ ]:
labels = data.iloc[:,1]
print(labels)

0        1
1        1
2        1
3        1
4        1
        ..
20123    1
20124    0
20125    1
20126    0
20127    1
Name: label, Length: 20128, dtype: int64


In [ ]:
model = Sequential()
embedding_layer = Embedding(input_dim=70,output_dim=16,input_length=256)
model.add(embedding_layer)
model.add(Reshape((1, 256, 16)))
model.add(layers.Conv2D(64, (1, 8), activation="relu", input_shape = (128, 16, 1)))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(16, (1,16), activation="relu"))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(8, (1,32), activation="relu"))
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1,  activation="sigmoid"))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 256, 16)           1120      
                                                                 
 reshape (Reshape)           (None, 1, 256, 16)        0         
                                                                 
 conv2d (Conv2D)             (None, 1, 249, 64)        8256      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 125, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 110, 16)        16400     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 55, 16)        0         
 2D)                                                    

In [ ]:
def embedding_urls(urls):
    vocab_size = 70
    encoded_urls = [one_hot(url,vocab_size) for url in urls]
#     print(f'Encoded URLs: {encoded_urls}')
    max_length = 256
    padded_urls = pad_sequences(encoded_urls,maxlen=max_length,padding='post')
    return padded_urls

In [ ]:
def evaluate_model(X_test_padded,y_test):
    loss, acc = model.evaluate(X_test_padded,y_test)
    return acc

In [ ]:
kf = KFold(n_splits=5)
accuracies = []

for train_index, test_index in kf.split(urls):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = urls[train_index], urls[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    X_train_padded = embedding_urls(X_train)
    X_test_padded = embedding_urls(X_test)
    model.fit(X_train_padded,y_train,batch_size = 64,epochs=100)
    accuracy = evaluate_model(X_test_padded,y_test)
    print(accuracy)
    accuracies.append(accuracy)

TRAIN: [ 4026  4027  4028 ... 20125 20126 20127] TEST: [   0    1    2 ... 4023 4024 4025]
Epoch 1/100
252/252 [==============================] - 15s 6ms/step - loss: 0.6297 - acc: 0.6272
Epoch 2/100
252/252 [==============================] - 1s 5ms/step - loss: 0.4424 - acc: 0.7918
Epoch 3/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3693 - acc: 0.8306
Epoch 4/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3470 - acc: 0.8443
Epoch 5/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3374 - acc: 0.8485
Epoch 6/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3304 - acc: 0.8493
Epoch 7/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3210 - acc: 0.8573
Epoch 8/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3135 - acc: 0.8574
Epoch 9/100
252/252 [==============================] - 1s 5ms/step - loss: 0.3050 - acc: 0.8660
Epoch 10/100
252/252 [======================

In [ ]:
print(accuracies)

[0.8539493083953857, 0.9234972596168518, 0.9418777823448181, 0.9515528082847595, 0.9619875550270081]


In [ ]:
print(sum(accuracies)/len(accuracies))

0.9265729427337647
